# Launcher test notebook

This notebook is a test notebook for running multiple simulations of magnetic mean field in Pencil code.

In [1]:
# Load autoreload extension for easier development of source code
%load_ext autoreload
%autoreload 2

## Get default parameters

In [2]:
import f90nml
defaults_run_in = f90nml.read('isotropic_alpha/run.in')

## Create function that defines the simulation

In [3]:
import sacred
import os
import shutil
from tools.launcher import PencilCodeLauncher
from tools.misc import copy_initial_simulation

         pencil has moved to pencil_old.
To change your scripts accordingly:
import pencilnew as pc -> import pencil as pc
import pencil as pc -> import pencil_old as pc


In [4]:
from sacred import Experiment
from sacred.observers import FileStorageObserver

ialpha_ex = Experiment(name='isotropic_alpha', interactive=True)

ialpha_ex.observers.append(FileStorageObserver('isotropic_alpha_runs'))

@ialpha_ex.config
def ialpha_config():
    nt = 5001
    dt = 2.5e-05
    alpha = 0.451
    
@ialpha_ex.capture
def ialpha_copy_initial_setup(alpha):
    print('Copying simulation directory for alpha={0}'.format(alpha))
    copy_initial_simulation('isotropic_alpha', 'isotropic_alpha-{0}'.format(alpha), overwrite=True)

@ialpha_ex.capture
def ialpha_template_run_in(alpha, nt, dt):
    print('Setting run.in for alpha={0}'.format(alpha))
    run_in = defaults_run_in.copy()
    
    run_in['run_pars']['nt'] = nt
    run_in['run_pars']['dt'] = dt
    run_in['special_run_pars']['alpha_scale'] = alpha
    run_in.write(os.path.join('isotropic_alpha-{0}'.format(alpha),'run.in'),
                 force=True)
    
@ialpha_ex.capture
def ialpha_start(alpha):
    print('Starting simulation for alpha={0}'.format(alpha))
    pl = PencilCodeLauncher('isotropic_alpha-{0}'.format(alpha), verbose=True)
    pl.start()

@ialpha_ex.capture
def ialpha_copy_coefs(alpha):
    print('Copying coefficients for alpha={0}'.format(alpha))
    shutil.copy('emftensors.h5',
                os.path.join('isotropic_alpha-{0}'.format(alpha),'data', 'emftensors.h5'))

@ialpha_ex.capture
def ialpha_run(alpha):
    print('Running simulation for alpha={0}'.format(alpha))
    pl = PencilCodeLauncher('isotropic_alpha-{0}'.format(alpha), verbose=True)
    pl.run()

@ialpha_ex.main
def ialpha_main():
    ialpha_copy_initial_setup()
    ialpha_template_run_in()
    ialpha_start()
    ialpha_copy_coefs()
    run_result = ialpha_run()
    return run_result

## Single run using sacred

In [5]:
ialpha_ex.run(config_updates={'alpha': 0.2, 'nt': 1001, 'dt': 2.5e-06})

INFO - isotropic_alpha - Running command 'ialpha_main'
INFO - isotropic_alpha - Started run with ID "2"
INFO - sh.command - <Command '/bin/bash -c pc_start', pid 5717>: process started


Copying simulation directory for alpha=0.2
Setting run.in for alpha=0.2
Starting simulation for alpha=0.2
Current state:
{'start': False, 'run': False}


INFO - sh.command - <Command '/bin/bash -c pc_run', pid 5773>: process started



Copying coefficients for alpha=0.2
Running simulation for alpha=0.2
Current state:
{'run': False, 'start': True}


INFO - isotropic_alpha - Completed after 0:00:12


## Multiple runs using pandas

In [6]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.dataframe.utils import make_meta

In [7]:
ialpha_sims = pd.DataFrame({'alpha':np.arange(0.4,0.651,0.05)})
ialpha_sims

,alpha
0,0.40
1,0.45
2,0.50
3,0.55
4,0.60
5,0.65


In [8]:
ialpha_sims['nt'] = int(1000)
ialpha_sims['dt'] = 1e-06

In [9]:
ialpha_sims

,alpha,nt,dt
0,0.40,1000,0.000001
1,0.45,1000,0.000001
2,0.50,1000,0.000001
3,0.55,1000,0.000001
4,0.60,1000,0.000001
5,0.65,1000,0.000001


In [10]:
ialpha_sims.apply(lambda x: print(dict(x)), axis=1)

{'alpha': 0.4, 'nt': 1000.0, 'dt': 1e-06}
{'alpha': 0.45, 'nt': 1000.0, 'dt': 1e-06}
{'alpha': 0.5, 'nt': 1000.0, 'dt': 1e-06}
{'alpha': 0.55, 'nt': 1000.0, 'dt': 1e-06}
{'alpha': 0.6, 'nt': 1000.0, 'dt': 1e-06}
{'alpha': 0.6499999999999999, 'nt': 1000.0, 'dt': 1e-06}


0    None
1    None
2    None
3    None
4    None
5    None
dtype: object

In [11]:
ialpha_sims['run'] = ialpha_sims.apply(lambda x: ialpha_ex.run(config_updates=dict(x)), axis=1)

WARNING - root - Changed type of config entry "nt" from int to float64
INFO - isotropic_alpha - Running command 'ialpha_main'
INFO - isotropic_alpha - Started run with ID "3"
INFO - sh.command - <Command '/bin/bash -c pc_start', pid 5854>: process started


Copying simulation directory for alpha=0.4
Setting run.in for alpha=0.4
Starting simulation for alpha=0.4
Current state:
{'start': False, 'run': False}


INFO - sh.command - <Command '/bin/bash -c pc_run', pid 5908>: process started



Copying coefficients for alpha=0.4
Running simulation for alpha=0.4
Current state:
{'run': False, 'start': True}


INFO - isotropic_alpha - Completed after 0:00:02


...

WARNING - root - Changed type of config entry "nt" from int to float64
INFO - isotropic_alpha - Running command 'ialpha_main'
INFO - isotropic_alpha - Started run with ID "8"
INFO - sh.command - <Command '/bin/bash -c pc_start', pid 6397>: process started


Copying simulation directory for alpha=0.6499999999999999
Setting run.in for alpha=0.6499999999999999
Starting simulation for alpha=0.6499999999999999
Current state:
{'start': False, 'run': False}


INFO - sh.command - <Command '/bin/bash -c pc_run', pid 6452>: process started



Copying coefficients for alpha=0.6499999999999999
Running simulation for alpha=0.6499999999999999
Current state:
{'run': False, 'start': True}


INFO - isotropic_alpha - Completed after 0:00:02


In [12]:
ialpha_sims

,alpha,nt,dt,run
0,0.40,1000,0.000001,<sacred.run.Run object at 0x7f7d76f95160>
1,0.45,1000,0.000001,<sacred.run.Run object at 0x7f7d76f4a070>
2,0.50,1000,0.000001,<sacred.run.Run object at 0x7f7d76f42880>
3,0.55,1000,0.000001,<sacred.run.Run object at 0x7f7d76f4ab80>
4,0.60,1000,0.000001,<sacred.run.Run object at 0x7f7d7c72aa90>
5,0.65,1000,0.000001,<sacred.run.Run object at 0x7f7d76f000d0>
